In [1]:
import os, shutil
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from keras.utils import to_categorical, load_img, img_to_array, array_to_img
from sklearn.model_selection import train_test_split
from keras import backend as K

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

#### House Pricing

In [2]:
from keras.datasets import boston_housing

In [3]:
(train_data, train_targets), (test_data, test_targets) = \
    boston_housing.load_data()

In [4]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [5]:
train_data, val_data, train_targets, val_targets = train_test_split(
    train_data, train_targets, test_size=0.25) 

In [6]:
train_data.shape, val_data.shape

((303, 13), (101, 13))

In [7]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [12]:
model = build_model()
history = model.fit(train_data, train_targets,
                    validation_data=(val_data, val_targets),
                    epochs = 80, batch_size=16, verbose=1)

Epoch 1/80
19/19 [==============================] - 2s 21ms/step - loss: 499.3076 - mae: 20.4225 - val_loss: 393.7769 - val_mae: 18.3700
Epoch 2/80
19/19 [==============================] - 0s 7ms/step - loss: 356.0129 - mae: 16.8792 - val_loss: 249.8304 - val_mae: 14.3094
Epoch 3/80
19/19 [==============================] - 0s 7ms/step - loss: 207.4306 - mae: 12.3210 - val_loss: 126.4600 - val_mae: 9.7938
Epoch 4/80
19/19 [==============================] - 0s 7ms/step - loss: 103.5346 - mae: 8.1277 - val_loss: 60.8776 - val_mae: 6.6089
Epoch 5/80
19/19 [==============================] - 0s 5ms/step - loss: 56.7229 - mae: 5.6838 - val_loss: 37.2532 - val_mae: 4.9349
Epoch 6/80
19/19 [==============================] - 0s 5ms/step - loss: 38.5712 - mae: 4.5441 - val_loss: 25.4592 - val_mae: 4.0780
Epoch 7/80
19/19 [==============================] - 0s 5ms/step - loss: 30.0832 - mae: 3.8343 - val_loss: 20.3774 - val_mae: 3.5522
Epoch 8/80
19/19 [==============================] - 0s 6ms/step

In [13]:
emv = pd.DataFrame({"emv": history.history['val_mae'][10:]}).ewm(com=2).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(y=emv["emv"], 
                         line=dict(color='royalblue', width=3)))
fig.show()